In [280]:
import serial
import time
import csv
import re
import matplotlib.pyplot as plt
import pandas as pd

In [281]:
portPath = "/dev/cu.usbmodem14101"       
baud = 9600                     
timeout = 5                       
filename = "sensorData.csv"
max_num_readings = 16000
num_signals = 1

In [282]:
def create_serial_obj(portPath, baud_rate, tout):
    """
    port path, baud rate, timeout value
    ––> return pyserial object
    """
    return serial.Serial(portPath, baud_rate, timeout = tout)

In [317]:
def read_serial_data(serial):
    """
    pyserial object (serial)
    ––> return list from serial port
    """
    serial.flushInput()
    
    serial_data = []
    readings_left = True
    timeout_reached = False
    
    counter = 0
    
    while readings_left and counter < 1000 and not timeout_reached:
        counter += 1
        serial_line = serial.readline().decode('utf-8') + str(time.time())
        if serial_line == '':
            timeout_reached = True
        else:
            serial_data.append(serial_line)
            if len(serial_data) == max_num_readings:
                readings_left = False
        
    return serial_data

In [318]:
def is_number(string):
    """
    input string 
    ––> returns true if  string is number.
    else returns false
    """
    try:
        float(string)
        return True
    except ValueError:
        return False

In [319]:
def clean_serial_data(data):
    """
    input list of serial lines and removes chars
    ––> return cleaned list of values
    eg: ['0.5000,33\r\n', '1.0000,283\r\n']
    ––> return [[0.5,33.0], [1.0,283.0]]
    """
    clean_data = []
    
    for line in data:
        line_data = re.findall("\d*\.\d*|\d*",line) 
        line_data = [float(element) for element in line_data if is_number(element)] 
        if len(line_data) >= 2:
            clean_data.append(line_data)
 
    return clean_data   

In [320]:
def save_to_csv(data, filename):
    """
    creates csv of sensor values to"sensorData.csv"
    """
    with open(filename, 'w') as csvfile:
        csvwrite = csv.writer(csvfile)
        csvwrite.writerows([data])

In [321]:
def gen_col_list(num_signals):
    """
    input number of signals 
    ––> returns list of the columns
    eg . 3 signals returns : ['Time','Signal1','Signal2','Signal3']
    """
    col_list = ['Time']
    for i in range(1,num_signals+1):
        col = 'Signal'+str(i)
        col_list.append(col)
        
    return col_list

In [322]:
def map_value(x, in_min, in_max, out_min, out_max):
    return (((x - in_min) * (out_max - out_min))/(in_max - in_min)) + out_min

In [323]:
%matplotlib inline
def simple_plot(csv_file, columns, headers):
    plt.clf()
    plt.close()
    plt.plotfile(csv_file, columns, names=headers, newfig=True)
    plt.xlabel('Time (sec)')
    plt.ylabel('Photoresistor values')
    plt.title('Photoresistor values over time')
    plt.show()

In [324]:
def plot_csv(csv_file, columns):
    data_frame = pd.read_csv(csv_file)
    data_frame.columns = columns
    data_frame = data_frame.set_index(cols[0])
    data_frame = data_frame.apply(lambda x: map_value(x,0.,1023,0,5))
    data_frame = data_frame.reset_index()
    
    plt.clf()
    plt.close()
    data_frame.plot(x=columns[0],y=columns[1:])
    plt.show()

In [325]:
print ("Creating serial object")
serial_obj = create_serial_obj(portPath, baud, timeout)

Creating serial object


In [326]:
print ("Reading serial data")
serial_data = read_serial_data(serial_obj)
##print (serial_data)
print("here")
print(serial_data[0])

Reading serial data
here
853
1571522899.4510279


In [315]:
print ("Cleaning data")
clean_data =  clean_serial_data(serial_data)

Cleaning data


In [327]:
print ("Saving to csv")
save_to_csv(serial_data, filename)

Saving to csv


In [304]:
print ("Plotting data...")
#simple_plot(filename, (0,1,2), ['time (s)', 'voltage1', 'voltage2'])
#simple_plot(filename, (0,1), ['time (s)', 'voltage1'])
plot_csv(filename, gen_col_list(num_signals))

Plotting data...


ValueError: Length mismatch: Expected axis has 10 elements, new values have 2 elements